In [ ]:
import pandas as pd

In [ ]:
df_dialogues_final = pd.read_csv('/content/drive/MyDrive/PROJECT/movie_screenplays/dialogues_data_final.csv')

In [ ]:
df_dialogues_final

,title,character,dialogue
0,A History of Violence,Billy,You think maybe we could just turn around
1,A History of Violence,Billy,shorter chunkier wearing jeans and an old John...
2,A History of Violence,Billy,Yeah I guess Im just tired though
3,A History of Violence,Billy,Johnny Leland humming along with Johnny slides...
4,A History of Violence,Billy,Yuh
...,...,...,...
490866,You Can Count on Me,Sammy,Please dont go till you know where youre going...
490867,You Can Count on Me,Terry,I do know where Im going Im going to Worcester...
490868,You Can Count on Me,Terry,Come on Sammy Look at me Look at me
490869,You Can Count on Me,Terry,Hey Sammy Remember when we were kids remember ...


In [ ]:
!pip install transformers

SENTIMENT CLASSIFICATION

In [ ]:
!rm -rf ~/.cache/huggingface/hub/arpanghoshal/EmoRoBERTa

In [ ]:
from transformers import pipeline
sentiment_task=pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
df_dialogues_final = df_dialogues_final[df_dialogues_final['dialogue'].notna()]
df_dialogues_final['dialogue'] = df_dialogues_final['dialogue'].astype(str)

In [ ]:
from tqdm import tqdm

BATCH_SIZE = 32

df_dialogues_final = df_dialogues_final.copy()

valid_texts = df_dialogues_final['dialogue'].tolist()
valid_indices = df_dialogues_final.index.tolist()

df_dialogues_final['Sentiment_Value'] = None
error_ids = []

In [ ]:
for i in tqdm(range(0, len(valid_texts), BATCH_SIZE)):
    batch_texts = valid_texts[i:i + BATCH_SIZE]
    batch_indices = valid_indices[i:i + BATCH_SIZE]

    try:
        predictions = sentiment_task(
            batch_texts,
            batch_size=BATCH_SIZE,
            truncation=True)

        for idx, pred in zip(batch_indices, predictions):
            score = pred['score']
            if pred['label'] == 'negative':
                score = -score
            elif pred['label'] == 'neutral':
                score = 0

            df_dialogues_final.loc[idx, 'Sentiment_Value'] = score

    except RuntimeError as e:
        print(f"ERROR {i}: {e}")
        for idx in batch_indices:
            df_dialogues_final.loc[idx, 'Sentiment_Value'] = 'ERROR'
            error_ids.append(idx)

print(f"Błędy: {len(error_ids)}")

100%|██████████| 15305/15305 [20:08<00:00, 12.67it/s]

Błędy: 0


In [ ]:
df_dialogues_final

,title,character,dialogue,Sentiment_Value
0,A History of Violence,Billy,You think maybe we could just turn around,0
1,A History of Violence,Billy,shorter chunkier wearing jeans and an old John...,0
2,A History of Violence,Billy,Yeah I guess Im just tired though,-0.490806
3,A History of Violence,Billy,Johnny Leland humming along with Johnny slides...,0
4,A History of Violence,Billy,Yuh,0
...,...,...,...,...
490866,You Can Count on Me,Sammy,Please dont go till you know where youre going...,0
490867,You Can Count on Me,Terry,I do know where Im going Im going to Worcester...,0
490868,You Can Count on Me,Terry,Come on Sammy Look at me Look at me,0.661844
490869,You Can Count on Me,Terry,Hey Sammy Remember when we were kids remember ...,0


In [ ]:
df_dialogues_final.to_csv('/content/drive/MyDrive/PROJECT/movie_screenplays/dialogues_data_final_sentiment.csv', index=False, encoding="utf-8-sig")

EMOTIONS CLASSIFICATION

In [ ]:
df_dialogues_final_sentiment = pd.read_csv('/content/drive/MyDrive/PROJECT/movie_screenplays/dialogues_data_final_sentiment.csv')

In [ ]:
from transformers import pipeline
classifier=pipeline("text-classification", model='bhadresh-savani/roberta-base-emotion', return_all_scores=True)

config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
BATCH_SIZE = 32

valid_texts = df_dialogues_final_sentiment['dialogue'].tolist()
valid_indices = df_dialogues_final_sentiment['dialogue'].index.tolist()

emotion_labels = ['Anger', 'Fear', 'Joy', 'Love', 'Sadness', 'Surprise']
for label in emotion_labels:
    if label not in df_dialogues_final_sentiment.columns:
        df_dialogues_final_sentiment[label] = None

In [ ]:
df_dialogues_final_sentiment[0:5]

,title,character,dialogue,Sentiment_Value,Anger,Fear,Joy,Love,Sadness,Surprise
0,A History of Violence,Billy,You think maybe we could just turn around,0.000000,None,None,None,None,None,None
1,A History of Violence,Billy,shorter chunkier wearing jeans and an old John...,0.000000,None,None,None,None,None,None
2,A History of Violence,Billy,Yeah I guess Im just tired though,-0.490806,None,None,None,None,None,None
3,A History of Violence,Billy,Johnny Leland humming along with Johnny slides...,0.000000,None,None,None,None,None,None
4,A History of Violence,Billy,Yuh,0.000000,None,None,None,None,None,None


In [ ]:
error_ids = []

for i in tqdm(range(0, len(valid_texts), BATCH_SIZE)):
    batch_texts = valid_texts[i:i + BATCH_SIZE]
    batch_indices = valid_indices[i:i + BATCH_SIZE]

    try:
        predictions = classifier(batch_texts)

        for idx, emotions in zip(batch_indices, predictions):
            for emotion in emotions:
                label = emotion['label'].capitalize()
                score = emotion['score']
                df_dialogues_final_sentiment.loc[idx, label] = score

    except Exception as e:
        print(f"Błąd w batch {i}: {e}")
        for idx in batch_indices:
            error_ids.append(idx)
            for label in emotion_labels:
                df_dialogues_final_sentiment.loc[idx, label] = 'ERROR'

print(f"Błędy: {len(error_ids)}")

100%|██████████| 15305/15305 [1:20:33<00:00,  3.17it/s]

Błędy: 0


In [ ]:
df_dialogues_final_sentiment

,title,character,dialogue,Sentiment_Value,Anger,Fear,Joy,Love,Sadness,Surprise
0,A History of Violence,Billy,You think maybe we could just turn around,0.000000,0.006424,0.006397,0.980991,0.003683,0.001661,0.000843
1,A History of Violence,Billy,shorter chunkier wearing jeans and an old John...,0.000000,0.070729,0.003774,0.794046,0.003214,0.127545,0.000693
2,A History of Violence,Billy,Yeah I guess Im just tired though,-0.490806,0.016421,0.001383,0.006594,0.000522,0.974675,0.000404
3,A History of Violence,Billy,Johnny Leland humming along with Johnny slides...,0.000000,0.120496,0.007768,0.847967,0.015595,0.007162,0.001013
4,A History of Violence,Billy,Yuh,0.000000,0.654012,0.10302,0.221834,0.004498,0.01381,0.002827
...,...,...,...,...,...,...,...,...,...,...
489743,You Can Count on Me,Sammy,Please dont go till you know where youre going...,0.000000,0.009457,0.004155,0.980754,0.002881,0.002474,0.000279
489744,You Can Count on Me,Terry,I do know where Im going Im going to Worcester...,0.000000,0.0029,0.001282,0.98914,0.000891,0.005543,0.000244
489745,You Can Count on Me,Terry,Come on Sammy Look at me Look at me,0.661844,0.738061,0.009986,0.24132,0.003172,0.006337,0.001124
489746,You Can Count on Me,Terry,Hey Sammy Remember when we were kids remember ...,0.000000,0.165478,0.005948,0.803718,0.00721,0.01639,0.001257


In [ ]:
df_dialogues_final_sentiment.to_csv('/content/drive/MyDrive/PROJECT/movie_screenplays/dialogues_data_final_sentiment_emotion.csv', index=False, encoding="utf-8-sig")